# The Graph Analyzer 
### With image processing and pdf input

In [ ]:
from opencv_alvin import *
import fitz

## pip install PyMuPDF


In [6]:

doc = fitz.open("animals.pdf")
for i in range(len(doc)):
    for img in doc.getPageImageList(i):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.writePNG("pdfImages/p%s-%s.png" % (i, xref))
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.writePNG("p%s-%s.png" % (i, xref))
            pix1 = None
        pix = None

## http://code.activestate.com/recipes/580703-extract-images-of-a-pdf-optionally-by-page-using-p/

### `After this all images are extracted within the pdfImages folder with proper numbering`

# feature detector

In [35]:
img = cv2.imread('bar_graph_images/basic.png',0)
#img = cv2.imread('mobile_phone/mobile_template.jpg',0)
orb = cv2.ORB_create()
# find the keypoints with ORB
kp = orb.detect(img,None)
# compute the descriptors with ORB
kp, des = orb.compute(img, kp)
img_keyPoints = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)

im(img)

In [74]:
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints with ORB
kp = orb.detect(img,None)
# compute the descriptors with ORB
kp, des = orb.compute(img, kp)
# draw only keypoints location,not size and orientation
img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
im(img2)

In [36]:
img_test = cv2.imread('bar_graph_images/bar1.jpg',0)
#img_test= cv2.imread('mobile_phone/mobile_in_hand.jpg',0)
kp_test = orb.detect(img_test,None)
# compute the descriptors with ORB
kp_test, des_test = orb.compute(img_test, kp_test)
img_test_keyPoints = cv2.drawKeypoints(img_test, kp_test, None, color=(0,255,0), flags=0)
im(img_test_keyPoints)

In [31]:
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des,des_test)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img3 = cv2.drawMatches(img,kp,img_test,kp_test,matches[:10],None,flags=2)
im(img3)

# bar graph detection using contours

In [138]:
img_org = cv2.imread('bar_graph_images/bar1.jpg')
img_gray=cv2.cvtColor(img_org,cv2.COLOR_BGR2GRAY)
#ret,thres=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
#dilated_img=cv2.dilate(img,(5,5))
kernel=np.ones((5,5),np.uint8)
img=cv2.morphologyEx(img_gray,cv2.MORPH_CLOSE,kernel)

im(img)


In [139]:
canny_edges=cv2.Canny(img,30,200)
im(canny_edges)


In [140]:
ci,contour,hierarchy=cv2.findContours(canny_edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#final_cont_img=cv2.drawContours(img_org,contour,-1,(255,0,0),1)
#im(final_cont_img)

In [141]:
count=0
disp=img_org
tot_len=len(contour)
for c in contour:
    
    accuracy=0.03*cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,accuracy,True)
    if len(approx)==4:
        count+=1
        disp=cv2.drawContours(disp,[c],0,(255,0,0),2)
    else:
        disp=cv2.drawContours(disp,[c],0,(0,0,255),2)
        print(len(approx))
print(str("accuracy is : ")+str(count/tot_len)+str(" ")+str(tot_len))
im(disp)

2
2
2
2
2
2
2
2
2
2
6
accuracy is : 0.0 11


## harris corner detection

In [136]:
harris_corners=cv2.cornerHarris(img_gray,3,3,0.06)

im(harris_corners)

In [161]:
import pytesseract as pyt
def remove_text(img):
    h=img.shape[0]
    boxes=pyt.image_to_boxes(img)
    for b in boxes.splitlines():
        b = b.split(' ')
        charLen_x=int((int(b[3])-int(b[1]))*0.5)
        charLen_y=int((int(b[4])-int(b[2]))*0.5)
#         if charLen_x>50:
#             continue
        img = cv2.rectangle(img, (int(b[1])-charLen_x,h - int(b[2])+charLen_y), (int(b[3])+charLen_x, h - int(b[4])-charLen_y), (255, 255, 255), -1)
    return img

In [169]:
bar2=cv2.imread('bar_graph_images/bar2.jpg')
im(bar2)
bar2_gray=cv2.cvtColor(bar2,cv2.COLOR_BGR2GRAY)
bar2_gray=remove_text(bar2_gray)

In [168]:
kernel=np.ones((5,5),np.uint8)
img=cv2.morphologyEx(bar2_gray,cv2.MORPH_CLOSE,kernel)
im(img)